Mike Cresswell: Data Pre-processing

In [12]:
#Mike Cresswell
#TCSS 555
#Pre-Processing

from __future__ import print_function
import nltk
from nltk.stem import *
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from collections import defaultdict
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from string import ascii_lowercase
from google.colab import files
import spacy
import requests
import gensim.downloader as api
import os
import io
import __future__
import re 
import numpy
import csv
import pandas as pd
import numpy as np
import math
import sys
import itertools

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [16]:
#Single step functions
def removeSpecialCharacterToken(words):
    somelist = [x for x in words if not IsSpecialCharacterToken(x)]
    return somelist

def IsSpecialCharacterWord(word):
    symbols = ['`','~','!','@','#','$','%','^','&','*','(',')','_','-','+','=','{','[','}','}','|','\\',':',';','"','\'','<',',','>','.','?','/','""',' ','']
    for x in symbols:
        if word == x:
            return True
    return False

def IsSpecialCharacterToken(word):
    newWord = re.sub('[^A-Za-z0-9]+', '', word)
    if word == '':
        return True
    return False

def checkWords(wordArray):
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    for x in wordArray: 
        if x not in english_vocab:
            print(x)
    return words

def stripWords(words):
    i = 0
    for x in words:   
        newWord = str.strip(x)
        newWord = str.strip(newWord, "`~!@#$%^&*()_-+={[}}|\\:;\"<,>.?/ ")
        newWord = re.sub('[^A-Za-z0-9]+', '', newWord)
        newWord = newWord.lower()
        words[i] = newWord
        i = i + 1
    return words


def getWords(line):
    words = []
    for w in line.split():
        words.append(w)
    return words

def Tokenize(words):
    words = word_tokenize(words);
    return words

def removeStopWords(words):
    stopWords = stopwords.words('english')
    somelist = [x for x in words if not IsStopWord(x,stopWords)]
    return somelist

def IsStopWord(word, stopWords):
    for x in stopWords:
        if word == x:
            return True
    return False

def removeNumbers(words):
    somelist = [x for x in words if not IsNumeric(x)]
    return somelist

def IsNumeric(word):
    if word.isnumeric():
        return True
    return False

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB }

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatizeWords(words):
    lemmatizer = WordNetLemmatizer()
    i = 0
    for x in words:           
        lemmizedWord = lemmatizer.lemmatize(x, get_wordnet_pos(x))
        words[i] = lemmizedWord
        i = i + 1
    return words

def stemmPorterWords(words):
    stemmer = PorterStemmer()
    i = 0
    for x in words: 
        stemmedWord = stemmer.stem(x)
        words[i] = stemmedWord
        i = i + 1
    return words


#Parent functions
#Use a number of single set functions to create cleaning/pre-processing algorithms

#Outputs a dataset with the text lemmatized
def processTextWithLemmanization(df, outputFilePath):
    lemmantizedData = []
    for index, row in df.iterrows(): 
        words = Tokenize(row['text'])
        words = stripWords(words)
        words = removeSpecialCharacterToken(words)
        words = removeStopWords(words)
        words = removeNumbers(words)
        lemmatizedWords = lemmatizeWords(words)
        newlemmatizedReview = " ".join(lemmatizedWords)
        d = {'id': row['id'], 'deceptive': row['deceptive'], 'hotel':row['hotel'], 'polarity':row['polarity'], 'source':row['source'], 'text':newlemmatizedReview }
        lemmantizedData.append(d)

    dfProcessedLemmatized = pd.DataFrame(data=lemmantizedData, columns=['id','deceptive','hotel','polarity','source','text'])
    dfProcessedLemmatized.to_csv(outputFilePath,index=False)
    dfProcessedLemmatized.head()

#Outputs a dataset with the text stemmed
def processTextWithStemming(df, outputFilePath):
    stemmedData = []
    for index, row in df.iterrows(): 
        words = Tokenize(row['text'])
        words = stripWords(words)
        words = removeSpecialCharacterToken(words)
        words = removeStopWords(words)
        words = removeNumbers(words)
        stemmedWords = stemmPorterWords(words)
        newStemmedReview = " ".join(stemmedWords)
        d = {'id': row['id'], 'deceptive': row['deceptive'], 'hotel':row['hotel'], 'polarity':row['polarity'], 'source':row['source'], 'text':newStemmedReview }
        stemmedData.append(d)
    
    dfProcessedStemmed = pd.DataFrame(data=stemmedData, columns=['id','deceptive','hotel','polarity','source','text'])
    dfProcessedStemmed.to_csv(outputFilePath,index=False)
    dfProcessedStemmed.head()

#Outputs a dataset without stemming or lemmaziation
def processText(df, outputFilePath):
    data = []
    for index, row in df.iterrows(): 
        words = Tokenize(row['text'])
        words = stripWords(words)
        words = removeSpecialCharacterToken(words)
        words = removeStopWords(words)
        words = removeNumbers(words)
        words = " ".join(words)
        d = {'id': row['id'], 'deceptive': row['deceptive'], 'hotel':row['hotel'], 'polarity':row['polarity'], 'source':row['source'], 'text':words }
        data.append(d)
    
    dfProcessed = pd.DataFrame(data=data, columns=['id','deceptive','hotel','polarity','source','text'])
    dfProcessed.to_csv(outputFilePath,index=False)
    dfProcessed.head()

In [17]:
#Driver Code
#Get Dataset
url="https://raw.githubusercontent.com/mgcresswell/TCSS555-Project/main/deceptive-opinion.csv"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8'))) 

#Call processing algorithms
path = os.getcwd()
processTextWithLemmanization(df, path+'\\deceptive-opinion_lemmatized.csv')
#processTextWithStemming(df, path+'\\deceptive-opinion_stemmed.csv')
#processText(df, path+'\\deceptive-opinion_processed.csv')

In [15]:
files.download(path+'\\deceptive-opinion_lemmatized.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>